# Get Article Abstracts

Try to find text and graphic abstracts for the publications in CDI

|Entries|Status |
|-------|-------|
|    468|None   |
|    180|Partial|
|     93|Full   |

In [23]:
# library containign read and write functions to csv file
import lib.handle_csv as csvh

# managing files and file paths
from pathlib import Path

# library for handling url searchs
import lib.handle_urls as urlh

# add a progress bar
from tqdm import tqdm_notebook
    
# library for accessing system functions
import os

# import custom functions (common to various notebooks)
import processing_functions as pr_fns

# Connecting to the db
import lib.handle_db as dbh

# Handle html requests 
import requests
from urllib.parse import urlparse

# just retrieve the whole talbe with column names
def get_table_data(db_name, table_name):
    
    db_conn = dbh.DataBaseAdapter(db_name)

    pubs = db_conn.get_full_table(table_name)

    pubs_ti = db_conn.get_table_info(table_name)

    pub_keys = [x[1] for x in pubs_ti]

    table_list = []
    for a_pub in pubs:
        res_dict ={}
        for a_key, a_val in zip(pub_keys, a_pub):
            res_dict[a_key] = a_val
        table_list.append(res_dict)
    return table_list

In [31]:
# get the publications list from the app database
db_file = "../cplas_data/dev_cplas_work.sqlite3"#"./db_files/production202505.sqlite3"


pub_list = get_table_data(db_file,'articles' )
print(f"The Graphical abstract link:\n\t{pub_list[21]['abstract']}")
print(f"The Graphical abstract link:\n\t{pub_list[21]['graphic_abstract']}")
print(pub_list[21]['url'])
print(pub_list[21]['link'])

The Graphical abstract link:
	The preservation of oral health over a person&#x27;s lifespan is a key factor for a high quality of life. Sustaining oral health requires high-end dental materials with a plethora of attributes such as durability, non-toxicity and ease of application. The combination of different requirements leads to increasing miniaturization and complexity of the material components such as the composite and adhesives, which makes the precise characterization of the material blend challenging. Here, we demonstrate how modern IR spectroscopy and imaging from the micro- to the nanoscale can provide insights on the chemical composition of the different material sections of a dental filling. We show how the recorded IR-images can be used for a fast and non-destructive porosity determination of the studied adhesive. Furthermore, the nanoscale study allows precise assessment of glass cluster structures and distribution within their characteristic organically modified ceramic 

In [32]:
import json
import time


transform_json_like = pub_list[21]['link'].replace('"=>', '":').replace('=>', ':')
url_data = json.loads(transform_json_like)
for a_url in url_data:
    if a_url['intended-application'] == "text-mining":
        print( a_url['URL'])
        get_abstract_use_url(a_url['URL']) 
        # for elsevier it redirects to a landing page with metadata:
        # the page contains two links:
        #  A link to self:
        #    <link href="https://api.elsevier.com/content/article/pii/S1742706123004026" rel="self"/>
        #  A link to the article landing page (the one we want to get to):
        #   <link href="https://www.sciencedirect.com/science/article/pii/S1742706123004026" rel="scidir"/>
        time.sleep(10)

https://api.elsevier.com/content/article/PII:S1742706123004026?httpAccept=text/xml
https://api.elsevier.com/
problem accessing 10.21468/scipostphys.17.6.149
<Response [200]>
https://api.elsevier.com/content/article/PII:S1742706123004026?httpAccept=text/plain
https://api.elsevier.com/
problem accessing 10.21468/scipostphys.17.6.149
<Response [429]>


In [21]:
abs_identifiers = {"nature":["Abs1-content"]}

def get_abstract_use_doi(doi_val):
    ret_txt = ""
    try:
        url_doi = "https://doi.org/" + doi_val
        r = requests.get(url_doi, allow_redirects=True) # Redirects help follow to the actual domain
        parsed_uri = urlparse(r.url) #url parse to get the scheme and domain name 
        result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
        print(result) # printing the result
        for abs_id in abs_identifiers:
            if abs_id in result:
                # try known tags for abstracts
                soup = bs.BeautifulSoup(r.content,'html')
                #print(soup.find("div",attrs={"class":"c-article-section__content","id":"aboutbook"}).text)
                #supa = soup.find(id="Abs1-content")
                ret_txt = str(soup.find("div", id="Abs1-content"))
            if ret_text != "":
                break
    except:
        print('problem accessing', an_article['doi'])
        print(r)
    
    return ret_txt
    results = []

def get_abstract_use_url(url_given):
    ret_txt = ""
    try:
        r = requests.get(url_given, allow_redirects=True) # Redirects help follow to the actual domain
        parsed_uri = urlparse(r.url) #url parse to get the scheme and domain name 
        result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
        print(result) # printing the result
        for abs_id in abs_identifiers:
            if abs_id in result:
                # try known tags for abstracts
                soup = bs.BeautifulSoup(r.content,'html')
                #print(soup.find("div",attrs={"class":"c-article-section__content","id":"aboutbook"}).text)
                #supa = soup.find(id="Abs1-content")
                ret_txt = str(soup.find("div", id="Abs1-content"))
            if ret_text != "":
                break
    except:
        print('problem accessing', an_article['doi'])
        print(r)
    
    return ret_txt
    results = []
    
for an_article in pub_list:
    try:
        if an_article['abstract'] == None: #or len (an_article['abstract']) < 700:
            print ("need abstract for", an_article['doi'])
            abstract_text = get_abstract_use_doi(an_article['doi'])
            print (abstract_text)
            if len(abstract_text) > 0:
                break
    except:
        print("Error on ", an_article['doi'],an_article['title'])  

need abstract for 10.1016/j.actbio.2023.07.017
https://linkinghub.elsevier.com/
problem accessing 10.1016/j.actbio.2023.07.017
<Response [200]>

need abstract for 10.1016/j.chemosphere.2024.143728
https://linkinghub.elsevier.com/
problem accessing 10.1016/j.chemosphere.2024.143728
<Response [200]>

need abstract for 10.1016/j.jcat.2024.115392
https://linkinghub.elsevier.com/
problem accessing 10.1016/j.jcat.2024.115392
<Response [200]>

need abstract for 10.1016/j.jphotochem.2023.114889
https://linkinghub.elsevier.com/
problem accessing 10.1016/j.jphotochem.2023.114889
<Response [200]>

need abstract for 10.1016/j.materresbull.2025.113352
https://linkinghub.elsevier.com/
problem accessing 10.1016/j.materresbull.2025.113352
<Response [200]>

need abstract for 10.1016/j.mattod.2023.09.009
https://linkinghub.elsevier.com/
problem accessing 10.1016/j.mattod.2023.09.009
<Response [200]>

need abstract for 10.1016/j.mtadv.2023.100361
https://linkinghub.elsevier.com/
problem accessing 10.1016

In [4]:
r.content

NameError: name 'r' is not defined

In [5]:
r.url

NameError: name 'r' is not defined

In [ ]:
import bs4 as bs

soup = bs.BeautifulSoup(r.content,'html')
#print(soup.find("div",attrs={"class":"c-article-section__content","id":"aboutbook"}).text)
#supa = soup.find(id="Abs1-content")
supa = soup.find( id="Abs1-content")

print(supa)



In [ ]:
type(str(supa))

In [ ]:
str(supa)

In [ ]:
res_dict ={}
for a_key, a_val in zip(pub_keys, pubs[0]):
    res_dict[a_key] = a_val


In [ ]:
res_dict

In [ ]:
zip(pub_keys, pubs[0])